# Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

если источник traffic_source равен yandex или google, то в source_type ставится organic

для источников paid и email из России - ставим ad

для источников paid и email не из России - ставим other

все остальные варианты берем из traffic_source без изменений

In [ ]:
import pandas as pd

log = pd.read_csv('visit_log.csv', sep=';')

log.loc[(log.traffic_source == 'yandex') | (log.traffic_source == 'google'), 'source_type'] = 'organic'
log.loc[((log.traffic_source == 'paid') | (log.traffic_source == 'email')) & (log.region == 'Russia'), 'source_type'] = 'ad'
log.loc[((log.traffic_source == 'paid') | (log.traffic_source == 'email')) & (log.region != 'Russia'), 'source_type'] = 'other'
log.head(20)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,NaN
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
5,1549980742,8855508aad,https://host.ru/df646c3676cc259fa0,Russia,fc43898e47,yandex,organic
6,1549980742,b0f66adc83,https://host.ru/b8b58337d272ee7b15,Russia,13fc55e781,paid,ad
7,1549980754,837885c8f8,https://host.ru/108ce4b365afb7b88e,Russia,cb5082b6f6,direct,NaN
8,1549980760,af5570f5a1,https://host.ru/3004a8273caeef2867,China,45664f7af2,direct,NaN
9,1549980765,3e7077fd2f,https://host.ru/df646c3676cc259fa0,Russia,6f9de8c8b6,email,ad


# Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

1. Прочитайте содержимое файла в датафрейм

2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [ ]:
df = pd.read_csv('URLs.txt')

df[df.url.str.contains(r'\w+\/\d{8}-\w+')].head(10)

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
10,/middleeast/36131117-divizion-s-400-ne-zametil...
11,/economics/36065674-rossiyane-vozmutilis-minim...
12,/politics/36118047-otets-24-detey-potreboval-i...
13,/travel/36194479-v-sheremetevo-passazhiry-ustr...
14,/politics/35638742-nesolidno-mariya-zaharova-s...


# Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [ ]:
import time

ratings = pd.read_csv('ratings.csv')

# датафрейм сгруппированный по userId с агрегатами: количество оценок и максимум/минимум по timestamp
ratings_grouped = ratings.groupby('userId').agg({'rating': 'count', 'timestamp': ['min', 'max']}) 

# ratings_grouped[(ratings_grouped['rating']['count'] > 100)] - фильтр по количеству оценок. Жизненный цикл: max timestamp - min timestamp.
lifetime = ratings_grouped[(ratings_grouped['rating']['count'] > 100)]['timestamp']['max'] - ratings_grouped[(ratings_grouped['rating']['count'] > 100)]['timestamp']['min']

print(f'Среднее время жизни - {lifetime.mean()}cек или {time.strftime("%H", time.gmtime(lifetime.mean()))}ч {time.strftime("%M", time.gmtime(lifetime.mean()))}мин {time.strftime("%S", time.gmtime(lifetime.mean()))}сек')

Среднее время жизни - 40080507.4496124cек или 21ч 28мин 27сек


# Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).
Необходимо сформировать две таблицы:

таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

аналогичную таблицу по типам выручки с указанием адреса клиента

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

all_revenue = rzd.merge(auto, how='outer').merge(air, how='outer').fillna(0) # таблица со всеми клиентами и всеми перевозками

all_revenue_address = all_revenue.merge(client_base) # таблица со всеми клиентами и всеми перевозками + адрес

print(all_revenue)

print(all_revenue_address)


   client_id  rzd_revenue  auto_revenue  air_revenue
0        111       1093.0           0.0          0.0
1        112       2810.0           0.0          0.0
2        113      10283.0       57483.0          0.0
3        114       5774.0          83.0          0.0
4        115        981.0         912.0         81.0
5        116          0.0        4834.0          4.0
6        117          0.0          98.0         13.0
7        118          0.0           0.0        173.0
   client_id  rzd_revenue  auto_revenue  air_revenue          address
0        111       1093.0           0.0          0.0  Комсомольская 4
1        112       2810.0           0.0          0.0   Энтузиастов 8а
2        113      10283.0       57483.0          0.0  Левобережная 1а
3        114       5774.0          83.0          0.0          Мира 14
4        115        981.0         912.0         81.0        ЗЖБИиДК 1
5        116          0.0        4834.0          4.0    Строителей 18
6        117          0.0        